# 05 Track Characteristics for 2026

The 2026 regulations are a complete reset. New aero, new power units, 30kg lighter cars. I can't use 2024-2025 lap times or team performance to predict anything.

But track geometry doesn't change. Monaco will still have tight corners, Monza will still have long straights. That's what I extract here.

When I get 2026 testing data showing which cars are fast where, we'll match those car profiles against these track profiles to predict which tracks suit which teams.

## Setup

In [1]:
import fastf1 as ff1
import pandas as pd
from pathlib import Path
import json

import logging
logging.getLogger("fastf1").setLevel(logging.ERROR)

import sys

PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

from src.helpers.circuit_utils import get_circuits
from src.helpers.track_extraction import (
    extract_track_profile,
    identify_street_circuits,
    calculate_track_z_scores,
    describe_track_profile
)

ff1.Cache.enable_cache('../data/raw/.fastf1_cache')

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

season = 2025

## Extract Track Geometry from 2025

Using qualifying sessions because they have clean laps and representative data.

Using helper functions from `src/helpers/track_extraction.py` for extraction.

### Querying the data

In [2]:
# Extract from all 2025 races
schedule = ff1.get_event_schedule(season)
races = schedule[schedule['EventFormat'] != 'testing']

track_profiles = {}
failed = []

for idx, race in races.iterrows():
    event_name = race['EventName']
    print(f"Processing {event_name}...")
    
    try:
        session = ff1.get_session(season, event_name, 'Q')
        session.load(telemetry=True, laps=True)
        
        profile = extract_track_profile(season, session)
        
        if profile:
            track_profiles[event_name] = profile
            print(f" 🟢 Done")
        else:
            failed.append(event_name)
            print(f" 🔴 Failed - no metrics")
            
    except Exception as e:
        failed.append(event_name)
        print(f" 🔴 Failed: {e}")

print(f"\n🟢 Extracted {len(track_profiles)} tracks")
if failed:
    print(f"🔴 Failed: {failed}")

Processing Australian Grand Prix...
 🟢 Done
Processing Chinese Grand Prix...
 🟢 Done
Processing Japanese Grand Prix...
 🟢 Done
Processing Bahrain Grand Prix...
 🟢 Done
Processing Saudi Arabian Grand Prix...
 🟢 Done
Processing Miami Grand Prix...
 🟢 Done
Processing Emilia Romagna Grand Prix...
 🟢 Done
Processing Monaco Grand Prix...
 🟢 Done
Processing Spanish Grand Prix...
 🟢 Done
Processing Canadian Grand Prix...
 🟢 Done
Processing Austrian Grand Prix...
 🟢 Done
Processing British Grand Prix...
 🟢 Done
Processing Belgian Grand Prix...
 🟢 Done
Processing Hungarian Grand Prix...
 🟢 Done
Processing Dutch Grand Prix...
 🟢 Done
Processing Italian Grand Prix...
 🟢 Done
Processing Azerbaijan Grand Prix...
 🟢 Done
Processing Singapore Grand Prix...
 🟢 Done
Processing United States Grand Prix...
 🟢 Done
Processing Mexico City Grand Prix...
 🟢 Done
Processing São Paulo Grand Prix...
 🟢 Done
Processing Las Vegas Grand Prix...
 🟢 Done
Processing Qatar Grand Prix...
 🟢 Done
Processing Abu Dhabi Gra

### Getting main characteristics of each circuit

In [3]:
# Add circuit metadata
circuits_season = get_circuits(2025)

for event_name, profile in track_profiles.items():
    # Match circuit info
    circuit_match = circuits_season[circuits_season['circuitName'].str.contains(event_name.split()[0], na=False)]
    
    if len(circuit_match) > 0:
        circuit = circuit_match.iloc[0]
        profile['altitude_m'] = circuit['altitude']
        profile['location'] = circuit['location']
        profile['country'] = circuit['country']

df_tracks = pd.DataFrame.from_dict(track_profiles, orient='index')
df_tracks

,track_name,slow_corner_pct,medium_corner_pct,fast_corner_pct,full_throttle_pct,top_speed_kmh,energy_score,braking_zones,extracted_from,avg_speed_loss_kmh,max_speed_loss_kmh,min_corner_speed_kmh,heavy_braking_pct,medium_braking_pct,light_braking_pct,total_corners,corner_density,altitude_m,location,country
Australian Grand Prix,Australian Grand Prix,0.046763,0.172662,0.780576,0.712230,329.0,4.096134,11,2025,100.333333,193.0,98.0,0.666667,0.166667,0.166667,6,1.151868,NaN,NaN,NaN
Chinese Grand Prix,Chinese Grand Prix,0.159639,0.237952,0.602410,0.575301,330.0,3.788902,8,2025,111.375000,263.0,65.0,0.625000,0.000000,0.375000,8,1.487735,NaN,NaN,NaN
Japanese Grand Prix,Japanese Grand Prix,0.074303,0.139319,0.786378,0.687307,325.0,3.857212,7,2025,100.500000,209.0,77.0,0.500000,0.250000,0.250000,8,1.383622,NaN,NaN,NaN
Bahrain Grand Prix,Bahrain Grand Prix,0.106383,0.273556,0.620061,0.595745,315.0,3.616952,10,2025,163.000000,248.0,67.0,0.875000,0.125000,0.000000,8,1.494668,8.0,Sakhir,Bahrain
Saudi Arabian Grand Prix,Saudi Arabian Grand Prix,0.050314,0.150943,0.798742,0.764151,338.0,4.072838,10,2025,106.714286,225.0,88.0,0.714286,0.000000,0.285714,7,1.147214,NaN,NaN,NaN
Miami Grand Prix,Miami Grand Prix,0.180685,0.205607,0.613707,0.613707,340.0,4.087835,10,2025,122.285714,271.0,69.0,0.428571,0.285714,0.285714,7,1.311169,5.0,Miami,USA
Emilia Romagna Grand Prix,Emilia Romagna Grand Prix,0.044280,0.254613,0.701107,0.678967,323.0,3.759230,9,2025,100.625000,180.0,104.0,0.625000,0.000000,0.375000,8,1.644897,NaN,NaN,NaN
Monaco Grand Prix,Monaco Grand Prix,0.315175,0.334630,0.350195,0.435798,289.0,2.817060,8,2025,61.363636,168.0,46.0,0.363636,0.181818,0.454545,11,3.356726,16.0,Monte Carlo,Monaco
Spanish Grand Prix,Spanish Grand Prix,0.022140,0.291513,0.686347,0.638376,328.0,3.716752,6,2025,122.666667,195.0,107.0,0.833333,0.000000,0.166667,6,1.296212,NaN,NaN,NaN
Canadian Grand Prix,Canadian Grand Prix,0.115672,0.276119,0.608209,0.664179,332.0,3.827475,15,2025,174.666667,232.0,65.0,1.000000,0.000000,0.000000,6,1.386826,NaN,NaN,NaN


In [4]:
# Identify street circuits
df_tracks['is_street_circuit'] = df_tracks.index.map(identify_street_circuits)

print(f"Street circuits identified: {df_tracks[df_tracks['is_street_circuit']==1].index.tolist()}")

Street circuits identified: ['Saudi Arabian Grand Prix', 'Miami Grand Prix', 'Monaco Grand Prix', 'Azerbaijan Grand Prix', 'Singapore Grand Prix', 'Las Vegas Grand Prix']


## Multi-Dimensional Track Profiling

Tracks aren't just "slow-corner" or "high-speed". They're somewhere in between on multiple dimensions.

Two corners at 120 km/h can be completely different:
- **Monaco T1**: Entry 180 km/h → Apex 60 km/h → Exit 110 km/h (120 km/h lost)
- **Silverstone Copse**: Entry 290 km/h → Apex 270 km/h → Exit 280 km/h (20 km/h lost)

Same speed classification, totally different demands. Monaco needs traction, Silverstone needs aero grip.

I capture multiple dimensions:
- **Corner speed** - What speed ranges the track operates in
- **Corner density** - How many corners per km (Monaco: ~9/km, Monza: ~2/km)
- **Tightest corner** - Slowest point on track (Monaco: ~50 km/h, most tracks: ~70+ km/h)
- **Corner severity** - How much braking demanded (Canada: 175 km/h avg loss, Monaco: 60 km/h)
- **Street circuit** - Binary flag for unique characteristics (tight, unforgiving, low grip)

I use z-scores to capture this. A z-score tells you how many standard deviations a track is from average:
- z = +2.0 means 2 std dev above average (extreme)
- z = 0.0 means exactly average
- z = -1.0 means 1 std dev below average

When I extract car characteristics from 2026 testing, we'll use the same z-score approach. Then I can mathematically match cars to tracks.

In [5]:
# Calculate z-scores for track characteristics
features = [
    # Corner speed distribution
    'slow_corner_pct',
    'medium_corner_pct',
    'fast_corner_pct',
    
    # Corner characteristics
    'corner_density',
    'min_corner_speed_kmh',
    'avg_speed_loss_kmh',
    'heavy_braking_pct',
    
    # Power characteristics
    'full_throttle_pct',
    
    # Tire stress
    'energy_score',
    
    # Complexity
    'braking_zones',
    
    # Track type
    'is_street_circuit'
]

# Calculate z-scores using helper function
df_tracks, scaler_params = calculate_track_z_scores(df_tracks, features)

print(f"Tracks with complete data: {len(df_tracks)}")
print("\nZ-score statistics:")
print(df_tracks[[f'{f}_z' for f in features]].describe())

Tracks with complete data: 24

Z-score statistics:
       slow_corner_pct_z  medium_corner_pct_z  fast_corner_pct_z  \
count       2.400000e+01         2.400000e+01       2.400000e+01   
mean       -1.110223e-16        -2.636780e-16      -2.960595e-16   
std         1.021508e+00         1.021508e+00       1.021508e+00   
min        -1.187742e+00        -1.843828e+00      -2.436019e+00   
25%        -7.043322e-01        -7.518308e-01      -4.713647e-01   
50%        -4.064149e-01         6.986027e-02      -6.386050e-02   
75%         4.908974e-01         5.630274e-01       9.784710e-01   
max         2.608641e+00         2.038885e+00       1.501392e+00   

       corner_density_z  min_corner_speed_kmh_z  avg_speed_loss_kmh_z  \
count      2.400000e+01               24.000000          2.400000e+01   
mean      -1.202742e-16                0.000000         -4.070818e-16   
std        1.021508e+00                1.021508          1.021508e+00   
min       -1.050222e+00               -2.171

## What Makes Each Track Unusual?

Looking at tracks with strong characteristics (z-score > 1.0)

In [6]:
# Generate track profile descriptions
df_tracks['profile_description'] = df_tracks.apply(describe_track_profile, axis=1)

# Show tracks with strong characteristics
print("Tracks with Distinctive Profiles:")
print("="*80)
for track in df_tracks.index:
    row = df_tracks.loc[track]
    if 'Balanced' not in row['profile_description']:
        print(f"\n{track}")
        print(f"  {row['profile_description']}")
        print(f"  Speed:     slow={row['slow_corner_pct_z']:+.2f}  med={row['medium_corner_pct_z']:+.2f}  fast={row['fast_corner_pct_z']:+.2f}")
        print(f"  Corners:   density={row['corner_density_z']:+.2f}  min_speed={row['min_corner_speed_kmh_z']:+.2f}")
        print(f"  Severity:  avg_loss={row['avg_speed_loss_kmh_z']:+.2f}  heavy_braking={row['heavy_braking_pct_z']:+.2f}")
        print(f"  Power:     throttle={row['full_throttle_pct_z']:+.2f}")
        print(f"  Tires:     energy={row['energy_score_z']:+.2f}")
        print(f"  Raw:       corner_density={row['corner_density']:.2f}/km  min_speed={row['min_corner_speed_kmh']:.0f} km/h")

Tracks with Distinctive Profiles:

Australian Grand Prix
  Heavy high-speed, High throttle demand
  Speed:     slow=-0.87  med=-0.90  fast=+1.09
  Corners:   density=-0.77  min_speed=+1.22
  Severity:  avg_loss=-0.36  heavy_braking=-0.06
  Power:     throttle=+1.08
  Tires:     energy=+0.86
  Raw:       corner_density=1.15/km  min_speed=98 km/h

Chinese Grand Prix
  Above-average slow corners
  Speed:     slow=+0.59  med=-0.01  fast=-0.37
  Corners:   density=-0.13  min_speed=-0.93
  Severity:  avg_loss=+0.04  heavy_braking=-0.32
  Power:     throttle=-0.56
  Tires:     energy=-0.03
  Raw:       corner_density=1.49/km  min_speed=65 km/h

Japanese Grand Prix
  Heavy high-speed
  Speed:     slow=-0.51  med=-1.36  fast=+1.14
  Corners:   density=-0.33  min_speed=-0.15
  Severity:  avg_loss=-0.35  heavy_braking=-1.09
  Power:     throttle=+0.78
  Tires:     energy=+0.16
  Raw:       corner_density=1.38/km  min_speed=77 km/h

Bahrain Grand Prix
  EXTREME braking demands, Traction-limited
  

## Track Rankings

Simple sorted lists for quick reference.

In [7]:
print("SLOW-CORNER RANKING (Top 8):")
print("-" * 80)
top_slow = df_tracks.nlargest(8, 'slow_corner_pct')
for i, (track, row) in enumerate(top_slow.iterrows(), 1):
    print(f"{i}. {track:30s} {row['slow_corner_pct']:6.1%} (z={row['slow_corner_pct_z']:+.2f})")

print("\nMEDIUM-CORNER RANKING (Top 8):")
print("-" * 80)
top_med = df_tracks.nlargest(8, 'medium_corner_pct')
for i, (track, row) in enumerate(top_med.iterrows(), 1):
    print(f"{i}. {track:30s} {row['medium_corner_pct']:6.1%} (z={row['medium_corner_pct_z']:+.2f})")

print("\nFAST-CORNER RANKING (Top 8):")
print("-" * 80)
top_fast = df_tracks.nlargest(8, 'fast_corner_pct')
for i, (track, row) in enumerate(top_fast.iterrows(), 1):
    print(f"{i}. {track:30s} {row['fast_corner_pct']:6.1%} (z={row['fast_corner_pct_z']:+.2f})")

print("\nCORNER DENSITY RANKING (Top 8 - most corners per km):")
print("-" * 80)
top_density = df_tracks.nlargest(8, 'corner_density')
for i, (track, row) in enumerate(top_density.iterrows(), 1):
    print(f"{i}. {track:30s} {row['corner_density']:5.2f} corners/km (z={row['corner_density_z']:+.2f})")

print("\nTIGHTEST CORNER RANKING (Top 8 - slowest apex speed):")
print("-" * 80)
top_tight = df_tracks.nsmallest(8, 'min_corner_speed_kmh')
for i, (track, row) in enumerate(top_tight.iterrows(), 1):
    print(f"{i}. {track:30s} {row['min_corner_speed_kmh']:5.0f} km/h min (z={row['min_corner_speed_kmh_z']:+.2f})")

print("\nCORNER SEVERITY RANKING (Top 8 - most speed lost):")
print("-" * 80)
top_severity = df_tracks.nlargest(8, 'avg_speed_loss_kmh')
for i, (track, row) in enumerate(top_severity.iterrows(), 1):
    print(f"{i}. {track:30s} {row['avg_speed_loss_kmh']:5.1f} km/h avg loss (z={row['avg_speed_loss_kmh_z']:+.2f})")

print("\nTRACTION-LIMITED RANKING (Top 8 - most heavy braking corners):")
print("-" * 80)
top_traction = df_tracks.nlargest(8, 'heavy_braking_pct')
for i, (track, row) in enumerate(top_traction.iterrows(), 1):
    print(f"{i}. {track:30s} {row['heavy_braking_pct']:5.1%} heavy corners (z={row['heavy_braking_pct_z']:+.2f})")

print("\nFULL THROTTLE RANKING (Top 8):")
print("-" * 80)
top_throttle = df_tracks.nlargest(8, 'full_throttle_pct')
for i, (track, row) in enumerate(top_throttle.iterrows(), 1):
    print(f"{i}. {track:30s} {row['full_throttle_pct']:6.1%} (z={row['full_throttle_pct_z']:+.2f})")

print("\nTIRE STRESS RANKING (Top 8):")
print("-" * 80)
top_energy = df_tracks.nlargest(8, 'energy_score')
for i, (track, row) in enumerate(top_energy.iterrows(), 1):
    print(f"{i}. {track:30s} energy={row['energy_score']:5.2f} (z={row['energy_score_z']:+.2f})")

print("\nSTREET CIRCUITS:")
print("-" * 80)
street = df_tracks[df_tracks['is_street_circuit'] == 1]
for track in street.index:
    print(f"  {track}")

SLOW-CORNER RANKING (Top 8):
--------------------------------------------------------------------------------
1. Monaco Grand Prix               31.5% (z=+2.61)
2. Las Vegas Grand Prix            27.2% (z=+2.05)
3. Mexico City Grand Prix          24.2% (z=+1.66)
4. Miami Grand Prix                18.1% (z=+0.87)
5. Azerbaijan Grand Prix           18.0% (z=+0.85)
6. Chinese Grand Prix              16.0% (z=+0.59)
7. Singapore Grand Prix            14.9% (z=+0.46)
8. United States Grand Prix        14.7% (z=+0.42)

MEDIUM-CORNER RANKING (Top 8):
--------------------------------------------------------------------------------
1. Singapore Grand Prix            38.8% (z=+2.04)
2. Hungarian Grand Prix            38.5% (z=+2.00)
3. Monaco Grand Prix               33.5% (z=+1.31)
4. Dutch Grand Prix                30.3% (z=+0.87)
5. Spanish Grand Prix              29.2% (z=+0.72)
6. Azerbaijan Grand Prix           28.6% (z=+0.65)
7. Mexico City Grand Prix          27.8% (z=+0.53)
8. Canadian 

## Save Track Database

Saving both raw percentages and z-scores. When I extract car characteristics from 2026 testing, we'll compute z-scores the same way, then match cars to tracks mathematically.

In [8]:

# Save
output_dir = Path('../data/processed/testing_files/track_characteristics')
output_dir.mkdir(exist_ok=True)

In [9]:
output_file = output_dir / f'{season}_track_characteristics.json'

# Prepare data for saving
output_data = {
    'metadata': {
        'extracted_from': season,
        'session_type': 'Qualifying',
        'num_tracks': len(df_tracks),
        'features': features,
        'scaler_params': scaler_params
    },
    'tracks': {}
}

for track in df_tracks.index:
    row = df_tracks.loc[track]
    output_data['tracks'][track] = {
        # Raw percentages
        'slow_corner_pct': float(row['slow_corner_pct']),
        'medium_corner_pct': float(row['medium_corner_pct']),
        'fast_corner_pct': float(row['fast_corner_pct']),
        'full_throttle_pct': float(row['full_throttle_pct']),
        'energy_score': float(row['energy_score']),
        'braking_zones': int(row['braking_zones']),
        'top_speed_kmh': float(row['top_speed_kmh']),
        
        # Corner characteristics
        'corner_density': float(row['corner_density']),
        'min_corner_speed_kmh': float(row['min_corner_speed_kmh']),
        'avg_speed_loss_kmh': float(row['avg_speed_loss_kmh']),
        'max_speed_loss_kmh': float(row['max_speed_loss_kmh']),
        'heavy_braking_pct': float(row['heavy_braking_pct']),
        'medium_braking_pct': float(row['medium_braking_pct']),
        'light_braking_pct': float(row['light_braking_pct']),
        'total_corners': int(row['total_corners']),
        
        # Track type
        'is_street_circuit': int(row['is_street_circuit']),
        
        # Z-scores (for car-track matching)
        'slow_corner_z': float(row['slow_corner_pct_z']),
        'medium_corner_z': float(row['medium_corner_pct_z']),
        'fast_corner_z': float(row['fast_corner_pct_z']),
        'corner_density_z': float(row['corner_density_z']),
        'min_corner_speed_z': float(row['min_corner_speed_kmh_z']),
        'avg_speed_loss_z': float(row['avg_speed_loss_kmh_z']),
        'heavy_braking_z': float(row['heavy_braking_pct_z']),
        'full_throttle_z': float(row['full_throttle_pct_z']),
        'energy_z': float(row['energy_score_z']),
        'braking_zones_z': float(row['braking_zones_z']),
        'is_street_circuit_z': float(row['is_street_circuit_z']),
        
        # Metadata
        'altitude_m': float(row['altitude_m']) if pd.notna(row.get('altitude_m')) else None,
        'location': row.get('location', None),
        'country': row.get('country', None),
        'profile': row['profile_description']
    }

with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=2)

In [10]:
# Also save as CSV for easy viewing
csv_cols = [
    # Raw values
    'slow_corner_pct', 'medium_corner_pct', 'fast_corner_pct',
    'full_throttle_pct', 'energy_score', 'braking_zones', 'top_speed_kmh',
    'corner_density', 'min_corner_speed_kmh', 'avg_speed_loss_kmh', 'max_speed_loss_kmh',
    'heavy_braking_pct', 'total_corners', 'is_street_circuit',
    # Z-scores
    'slow_corner_pct_z', 'medium_corner_pct_z', 'fast_corner_pct_z',
    'corner_density_z', 'min_corner_speed_kmh_z', 'avg_speed_loss_kmh_z', 'heavy_braking_pct_z',
    'full_throttle_pct_z', 'energy_score_z', 'braking_zones_z', 'is_street_circuit_z',
    # Metadata
    'altitude_m', 'location', 'country', 'profile_description'
]
df_tracks[csv_cols].to_csv(output_dir / f'{season}_track_characteristics.csv')
print(f"Also saved CSV to {output_dir / f'{season}_track_characteristics.csv'}")

Also saved CSV to ../data/processed/testing_files/track_characteristics/2025_track_characteristics.csv


## Summary

Track database built with 11 key characteristics:

**Corner characteristics:**
1. Corner speed distribution (slow/medium/fast %)
2. Corner density (corners per km) - How twisty is the track?
3. Minimum corner speed - How tight are the tightest corners?
4. Average speed loss - How hard are the braking zones?
5. Heavy braking % - What % of corners are traction-limited?

**Power & tire:**
6. Full throttle % - Power delivery demands
7. Energy score - Tire stress proxy

**Other:**
8. Braking zones - Complexity
9. Street circuit flag - Different characteristics (tight, unforgiving, low grip)

All characteristics stored as z-scores for car-track matching.

**Key insights:**
- Corner density captures "twisty" better than slow-corner %
- Min corner speed captures "tight" better than average speed
- Street circuits have unique demands beyond just corner types